> rank, alpha, dropout 설정은 더 찾아봐야 해요

> Epoch설정도 validation이랑 같이 돌려서 설정해야 함요.

In [ ]:
df = pd.read_csv("/content/train_unique_df.csv")

In [ ]:
df

,id,title,author,translator,publish_year,period,origin_text,sentence_count
0,P0991-01-47,벼락,빅토르 위고,윤세홍,2018,근대,"나는 가끔 하늘에서 거대한 붉은빛이 내게로 달려드는, 이 엄숙하고 놀라운 일을 겪는다.",6
1,P0991-01-47,벼락,빅토르 위고,윤세홍,2018,근대,"오, 사색가여, 그대처럼, 이사야의 슬픈 마음이 노엽게 만들고 증오했던, 숭고한 벼...",6
2,P0991-01-47,벼락,빅토르 위고,윤세홍,2018,근대,"섬광은 내 외침 소리, 벼락은 내 목소리.",6
3,P0991-01-47,벼락,빅토르 위고,윤세홍,2018,근대,"그대가 왕들에게 하는 것처럼, 나는 암초에 대고 노호한다.",6
4,P0991-01-47,벼락,빅토르 위고,윤세홍,2018,근대,"나는 무섭고 갑작스러운, 하늘나라의 경적.",6
...,...,...,...,...,...,...,...,...
9487,P1290-01-50,쓸쓸한 시절,이상화/이장희,장현숙,2014,현대,어느덧 가을은 깊어 들이든 뫼이든 숲이든 모두 파리해 있다.,5
9488,P1290-01-50,쓸쓸한 시절,이상화/이장희,장현숙,2014,현대,언덕 우에 오뚝히 서서 개가 짖는다.,5
9489,P1290-01-50,쓸쓸한 시절,이상화/이장희,장현숙,2014,현대,날카롭게 짖는다.,5
9490,P1290-01-50,쓸쓸한 시절,이상화/이장희,장현숙,2014,현대,빈 들에 마른 잎 태우는 연기 가늘게 가늘게 떠오른다.,5


In [ ]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# KoBERT 모델과 토크나이저 로드
model = BertModel.from_pretrained('monologg/kobert').to(device)
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# 모든 파라미터 동결
for param in model.parameters():
    param.requires_grad = False

# LoRA 레이어 정의
class LoRALayer(nn.Module):
    def __init__(self, input_dim, rank=8, alpha=16, dropout=0.05):
        super(LoRALayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = input_dim
        self.down_project = nn.Linear(input_dim, rank, bias=False)
        self.up_project = nn.Linear(rank, input_dim, bias=False)
        self.dropout = nn.Dropout(p=dropout)  # Dropout 추가


        # Scaling Factor 추가
        self.alpha = alpha
        self.scaling = self.alpha / rank  # 스케일링 계산

    def forward(self, x):
        x = self.down_project(x)  # 차원 축소
        x = self.dropout(x)       # Dropout 적용
        return self.scaling * self.up_project(x)  # 스케일링 및 차원 복원

# LoRA 레이어 초기화
lora_layer = LoRALayer(input_dim=768, rank=16).to(device)

# 데이터셋 정의
class LiteraryMLMDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512, mask_prob=0.15):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_prob = mask_prob

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        input_ids = encoding['input_ids'].squeeze()

        # MLM: 마스킹 생성
        labels = input_ids.clone()
        rand = torch.rand(input_ids.shape)
        mask_arr = (rand < self.mask_prob) * (input_ids != self.tokenizer.cls_token_id) * \
                   (input_ids != self.tokenizer.sep_token_id) * (input_ids != self.tokenizer.pad_token_id)

        # 마스킹된 위치를 [MASK] 토큰으로 대체
        input_ids[mask_arr] = self.tokenizer.mask_token_id

        # 마스크된 위치의 정답은 그대로 남기고, 나머지는 -100으로 설정 (loss 계산 제외)
        labels[~mask_arr] = -100
        return {
            'input_ids': input_ids.to(device),
            'attention_mask': encoding['attention_mask'].squeeze().to(device),
            'labels': labels.to(device)
        }

# DataLoader 생성
texts = df['origin_text'].tolist()
dataset = LiteraryMLMDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 옵티마이저 설정 (LoRA 레이어만 학습)
vocab_size = tokenizer.vocab_size
prediction_layer = nn.Linear(lora_layer.output_dim, vocab_size).to(device)
optimizer = torch.optim.AdamW(
    list(lora_layer.parameters()) + list(prediction_layer.parameters()),
    lr=1e-4,
    weight_decay=0.1
)

# 학습 루프
model.train()
lora_layer.train()
prediction_layer.train()

for epoch in range(3):
    total_loss = 0
    i = 0
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # 프리트레인된 KoBERT 모델에서 임베딩 추출
        with torch.no_grad():  # KoBERT의 파라미터는 업데이트하지 않음
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pretrained_embedding = outputs.last_hidden_state  # [batch_size, sequence_length, hidden_size]

        # LoRA 레이어 통과
        lora_embedding = lora_layer(pretrained_embedding)

        # 프리트레인된 임베딩 +  LoRA 임베딩
        combined_embedding = pretrained_embedding + lora_embedding

        # 예측 layer에 위에서 계산된 임베딩 삽입
        predictions = prediction_layer(combined_embedding)

        # 손실 계산
        loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
        loss = loss_fn(predictions.view(-1, vocab_size), labels.view(-1))

        # 역전파 및 옵티마이저 단계
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        i += 1

        print(f"batch {i}'s loss: {loss}")

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

Using device: cuda


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


batch 1's loss: 8.875059127807617
batch 2's loss: 8.91606616973877
batch 3's loss: 8.828226089477539
batch 4's loss: 8.915282249450684
batch 5's loss: 8.873071670532227
batch 6's loss: 8.762006759643555
batch 7's loss: 8.749157905578613
batch 8's loss: 8.724837303161621
batch 9's loss: 8.664798736572266
batch 10's loss: 8.645851135253906
batch 11's loss: 8.636205673217773
batch 12's loss: 8.747366905212402
batch 13's loss: 8.552626609802246
batch 14's loss: 8.443209648132324
batch 15's loss: 8.456201553344727
batch 16's loss: 8.458111763000488
batch 17's loss: 8.383665084838867
batch 18's loss: 8.359495162963867
batch 19's loss: 8.439655303955078
batch 20's loss: 8.462103843688965
batch 21's loss: 8.376684188842773
batch 22's loss: 8.360631942749023
batch 23's loss: 8.236004829406738
batch 24's loss: 8.428470611572266
batch 25's loss: 8.143943786621094
batch 26's loss: 8.104316711425781
batch 27's loss: 8.103841781616211
batch 28's loss: 8.188817024230957
batch 29's loss: 7.77085208892

KeyboardInterrupt: 